1. Thu thập các đường dẫn thuộc mỗi chuyên mục từ trang web https://vnexpress.net/

- Viết hàm lấy tất cả các đường dẫn của bài báo trong trang web

In [1]:
# Gọi thư viện
from urllib.request import urlopen 
from bs4 import BeautifulSoup
import re

# Đường dẫn đến trang web gốc:
URL = 'https://vnexpress.net/'

def _get_links(url):
    links = [] 
    html = urlopen(url).read()  
    soup = BeautifulSoup(html, 'html.parser') 

    # Lấy tất cả đường dẫn trong trang web hiện tại
    urls = set(re.findall(r'https://vnexpress.net/[^\s"]+\.html', str(soup)))  

    for url in urls:
        # Chỉ lấy đường dẫn là trang web nhiều bài báo
        if url != URL and url.startswith(URL) and not url.startswith(URL + '/tac-gia'): 
            links.append(url)

    return links


- Kết quả trả về của hàm với đường dẫn gốc của VNexpress

In [2]:
_get_links(URL)

['https://vnexpress.net/dai-gia-la-quang-binh-bi-cao-buoc-hoi-lo-sep-ngan-hang-bang-co-phieu-4804265.html',
 'https://vnexpress.net/khoanh-khac-phao-nhiet-ap-nga-thieu-dot-cu-diem-ukraine-4804354.html',
 'https://vnexpress.net/con-sot-han-kang-chua-ha-nhiet-4804336.html',
 'https://vnexpress.net/starship-ten-lua-manh-nhat-co-the-thay-doi-ca-nganh-vu-tru-4804307.html',
 'https://vnexpress.net/ong-trump-dan-truoc-ba-harris-trong-bo-phieu-som-o-cac-bang-chien-truong-4804365.html',
 'https://vnexpress.net/trieu-tien-noi-co-bang-chung-han-quoc-dieu-drone-vao-binh-nhuong-4804397.html',
 'https://vnexpress.net/hon-150-may-bay-trung-quoc-tap-tran-quanh-dao-dai-loan-4804293.html',
 'https://vnexpress.net/duong-sat-nao-cho-hang-4803920.html',
 'https://vnexpress.net/thanh-vu-ve-thu-bay-giai-ultra-triathlon-dai-nhat-the-gioi-4804447.html',
 'https://vnexpress.net/chung-khoan-hom-nay-15-10-co-phieu-chung-khoan-gay-ap-luc-cho-thi-truong-4804426.html',
 'https://vnexpress.net/nguoi-dan-do-nha-giao-d

- Lấy đường dẫn của các bài toán thuộc từng chuyên mục

In [3]:
# Danh sách các chuyên mục
categories = [
    'thoi-su', 'kinh-doanh', 'khoa-hoc', 'giai-tri',
    'the-thao', 'phap-luat', 'giao-duc', 'suc-khoe',
    'doi-song', 'du-lich', 'so-hoa', 'xe'
]

# Danh sách các đường dẫn đến các bài báo của mỗi chuyên mục
article_url = {}

for category in categories:
    article_url[category] = []

    # Lấy 5 trang trong mỗi chuyên mục
    for i in range(1, 3):
        url = '{}/{}-p{}'.format(URL, category, str(i))
        article_url[category] += _get_links(url)

    # Loại bỏ đường dẫn trùng nhau
    article_url[category] = list(set(article_url[category]))

    # Hiển thị số đường dẫn thu thập được trong chuyên mục
    print(category, len(article_url[category]))



thoi-su 27
kinh-doanh 27
khoa-hoc 27
giai-tri 27
the-thao 27
phap-luat 27
giao-duc 27


URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>

In [11]:
# Lấy riêng các đường dẫn trong chuyên mục theo góc nhìn từng mục con
goc_nhin = [
    'binh-luan-nhieu', 'chinh-tri-chinh-sach', 'y-te-su-khoe',
    'kinh-doanh-quan-tri', 'giao-duc-tri-thuc', 'moi-truong',
    'van-hoa-loi-song', 'covid-19'
]

article_url['goc-nhin'] = []

for sub_cate in goc_nhin:
    url = '{}/{}/{}'.format(URL, 'goc-nhin', sub_cate)
    article_url['goc-nhin'] += _get_links(url)

# Loại bỏ đường dẫn trùng nhau trong chuyên mục 'goc-nhin'
article_url['goc-nhin'] = list(set(article_url['goc-nhin']))
print('goc-nhin', len(article_url['goc-nhin']))

goc-nhin 27


2. Lấy nội dung của bài báo ứng với từng đường dẫn cụ thể và lưu vào một file trong folder chuyên mục tương ứng.

- Viết hàm lấy nội dung của bài báo ứng với đường dẫn cụ thể

In [12]:
def _get_content(url):
    # Nội dung bài báo
    content = ''
    
    # Mở URL và đọc nội dung trang
    html = urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    # Lấy phần chứa nội dung bài báo
    div_content = soup.select('.page-detail .container')
    if len(div_content) > 0:
        div_content = div_content[0]

        # Lấy phần mô tả
        description = div_content.find_all('p', {'class': 'description'})
        if len(description) > 0:
            text_description = description[-1].get_text()
            location = description[-1].find('span', {'class': 'location-stamp'})
            if location is not None:
                content = text_description[len(location.get_text()):]
            else:
                content = text_description   

        # Lấy phần chứa nội dung chi tiết bài báo
        detail = div_content.find('article', {'class': 'fck_detail'})
        if detail is not None:
            p_normal = detail.find_all('p', {'class': 'Normal'})
            if len(p_normal) > 0:
                for p in p_normal:
                    p_text = p.get_text()
                    # Kiểm tra đoạn văn bản không rỗng, không bắt đầu bằng dấu '>>' và kết thúc bằng các ký tự hợp lệ
                    if p_text != '' and not p_text.startswith('>>') and p_text[-1] in ['.', '!', '?']:
                        content += ' ' + p_text

    # Xóa ký tự ngắt dòng và trả về
    return re.sub(r'\n', '', content)

+ Kết quả trả về của hàm với đường dẫn dầu tiên trong chuyên mục số hóa

In [13]:
_get_content(article_url['giao-duc'][0])

'Lã Quang Bình, Chủ tịch HĐQT Công ty ECPAY, bị cáo buộc chỉ đạo em gái hối lộ sếp ngân hàng 200.000 cổ phiếu để được tiếp tục giải ngân nhằm tháo gỡ khó khăn cho công ty. Ông Lã Quang Bình là Chủ tịch HĐQT của Công ty Cổ phần Giải pháp thanh toán Điện lực và Viễn thông (ECPAY), Đầu tư thương mại dịch vụ Điện lực (ECInvest), Tập đoàn LALUNA và khách sạn Bến Du Thuyền (Marina Hotel). Hồi tháng 9, ông Bình cùng em gái Lã Thị Phương Liên bị Cơ quan An ninh điều tra, Bộ Công an, đề nghị truy tố về hai tội Vi phạm quy định về hoạt động ngân hàng, hoạt động khác liên quan đến hoạt động ngân hàng và Đưa hối lộ. Trong vụ án này, Đào Hoàng Thắng (cựu Giám đốc Agribank chi nhánh quận Đống Đa) bị đề nghị truy tố tội Nhận hối lộ. Theo kết luận, từ 2005 đến 2016, ông Bình thành lập một số công ty hoạt động trong lĩnh vực công nghệ thông tin, máy vi tính. Các công ty của Bình làm đại lý trung gian thu hộ tiền điện theo hợp đồng đã ký với các công ty điện lực; dịch vụ trung gian thanh toán, kinh doan

- Lưu nội dung các bài báo thành các file nằm trong folder chuyên mục tương ứng

In [16]:
import os

# Danh sách tất cả các câu đã làm sạch của tất cả các bài viết
sentence = []
#Chữ cái in hoa cảu tiếng việt
uppercase = "AĂÂBCDĐEÊGHIKLMNOÔƠPQRSTUƯVXY"

for category in article_url.keys():
    # Tạo folder chuyên mục để chứa các file bài viết
    current_path = 'data/categories' + category + '/'
    os.mkdir(current_path)

# Lưu các bài viết thành từng file
    count = 0
    for i in range(len(article_url[category])):
        content = _get_content(article_url[category][i])
        if content != '':
            count += 1
            f = open('{}/{}_{}.txt'.format(current_path, category, str(count)), 'w', encoding='utf-8') 
            # Chèn khoảng cách vào sau các dấu câu (., ?, !) nếu không có khoảng trắng hoặc dấu ngoặc kép ngay sau đó
            lines = re.sub(r'(?<=[.?!])(?=[\'"{}])'.format(uppercase), ' ', content)

            # Tách bài viết thành câu (giả sử câu kết thúc bằng ., !, ? và câu mới bắt đầu bằng chữ cái in hoa hoặc chữ số)
            lines = re.split(r'(?<=[.?!])\s*(?=[\'"{}0-9])'.format(uppercase), content)


            # Ghi từng câu trong vào viết vào file
            for line in lines:
                f.write((line + '\n'))
                sentence.append(line)
            f.close
        # Sổ bài viết trong chuyên mục    
        print(category, count)

        sentence = list(set(sentence))
        print("Tổng số câu được tách: ", len(sentence))

thoi-su 1
Tổng số câu được tách:  39
thoi-su 2
Tổng số câu được tách:  40
thoi-su 3
Tổng số câu được tách:  88
thoi-su 4
Tổng số câu được tách:  168
thoi-su 5
Tổng số câu được tách:  186
thoi-su 6
Tổng số câu được tách:  202
thoi-su 7
Tổng số câu được tách:  218
thoi-su 8
Tổng số câu được tách:  265
thoi-su 9
Tổng số câu được tách:  287
thoi-su 10
Tổng số câu được tách:  323
thoi-su 11
Tổng số câu được tách:  346
thoi-su 12
Tổng số câu được tách:  392
thoi-su 13
Tổng số câu được tách:  402
thoi-su 14
Tổng số câu được tách:  432
thoi-su 15
Tổng số câu được tách:  451
thoi-su 16
Tổng số câu được tách:  472
thoi-su 17
Tổng số câu được tách:  480
thoi-su 17
Tổng số câu được tách:  480
thoi-su 18
Tổng số câu được tách:  481
thoi-su 19
Tổng số câu được tách:  539
thoi-su 20
Tổng số câu được tách:  564
thoi-su 21
Tổng số câu được tách:  595
thoi-su 22
Tổng số câu được tách:  608
thoi-su 23
Tổng số câu được tách:  639
thoi-su 24
Tổng số câu được tách:  666
thoi-su 25
Tổng số câu được tách:  71

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'data/categorieskinh-doanh/'

+ 10 câu dầu tiên tách được

In [17]:
sentence[:10]

['Thủ tướng Israel Benjamin Nentayahu ngày 13/10 bày tỏ lấy làm tiếc trước việc một số lính giữ gìn hòa bình LHQ bị thương, đồng thời yêu cầu phái bộ rút lực lượng "ngay lập tức" để tránh tiếp tục phải đối mặt với tình thế "nguy hiểm".',
 'Lã Quang Bình, Chủ tịch HĐQT Công ty ECPAY, bị cáo buộc chỉ đạo em gái hối lộ sếp ngân hàng 200.',
 'Hass nhấn mạnh hành động của Musk trong những tuần tới có thể tác động đáng kể đến kết quả bầu cử và bối cảnh chính trị Mỹ trong tương lai.',
 'Seoul cho biết Bình Nhưỡng đã kích nổ mìn phá hủy nhiều đoạn của tuyến đường xuyên biên giới bên phía Triều Tiên, khi căng thẳng hai nước gia tăng.',
 '000 mm) dẫn tới phải chuyển hàng sang đoàn tàu trên khổ tiêu chuẩn (1.',
 'Lực lượng này cho biết các tòa nhà của LHQ ở làng Ramyah cũng đã hứng "thiệt hại nghiêm trọng" do những vụ nổ gần đó.',
 'Ngoài nâng sức mạnh xử lý, iPad cỡ nhỏ của Apple cũng có điểm mới dành cho những người sáng tạo nội dung là hỗ trợ bút Pencil Pro.',
 'Han Kang thắng giải Nobel Văn h

<strong>3. Lưu tất cả các câu vào file ngữ liệu </strong>

In [18]:
# File ngữ liệu
file_corpus = open('data/corpus.txt', 'w', encoding='utf-8')

count = 0
for sent in sentence:
    # Xóa ký tự thừa
    sent = re.sub(r'\u200b', '', sent)

    file_corpus.write(sent.strip() + '\n')
    count +=1
file_corpus.close()

print('Số lượng câu trong file ngữ liệu: ', count)

Số lượng câu trong file ngữ liệu:  733
